In [1]:
import tkinter as tk
import os
import glob
from PIL import Image, ImageTk
import shutil

In [2]:
class ImageClassifier:
    def __init__(self, master, image_dir, output_dir_A, output_dir_B):
        self.master = master
        self.image_dir = image_dir
        self.output_dir_A = output_dir_A
        self.output_dir_B = output_dir_B
        self.images = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]
        self.total_images = len(self.images)
        self.current_index = 363
        self.image_label = None
        self.text_label = None
        self.move_history = []
        self.create_widgets()

    def create_widgets(self):
        # create buttons and labels
        self.button_A = tk.Button(self.master, text="TEXT \n (Space)", command=self.classify_A)
        self.button_B = tk.Button(self.master, text="NO TEXT \n (Enter)", command=self.classify_B)
        self.button_undo = tk.Button(self.master, text="Undo \n (<--)", command=self.undo)
        self.button_skip = tk.Button(self.master, text="Skip \n (-->)", command=self.skip)
        
        # Bind keyboard keys to the buttons
        self.master.bind('<Return>', lambda event: self.classify_B())
        self.master.bind('<space>', lambda event: self.classify_A())
        self.master.bind('<Left>', lambda event: self.undo())
        self.master.bind('<Right>', lambda event: self.skip())
        
        self.image_label = tk.Label(self.master)
        self.text_label = tk.Label(self.master)

        # layout using grid

        self.text_label.grid(row=1, column=1, columnspan=4)
        
        self.image_label.grid(row=2, column=1, columnspan=4)
        
        self.button_A.grid(row=3, column=1, pady=10)
        self.button_B.grid(row=3, column=4, pady=10)
        self.button_undo.grid(row=3, column=2, pady=10)
        self.button_skip.grid(row=3, column=3, pady=10)
        
        self.button_A.config(bg="white")
        self.button_B.config(bg="white")
        self.button_undo.config(bg="white")
        self.button_skip.config(bg="white")

        # load first image
        self.load_image()

    def load_image(self):
        if self.current_index < 0 or self.current_index >= len(self.images):
            return
        image_name = self.images[self.current_index]
        self.image = Image.open(os.path.join(self.image_dir,image_name))
        self.image = self.image.resize((800, 800), Image.ANTIALIAS)
        self.photo = ImageTk.PhotoImage(self.image)

        self.image_label.config(image=self.photo)        
        self.text_label.config(text="Progress: {}/{}".format(self.current_index + 1, self.total_images))

    def classify_A(self):
        self.button_A.config(bg="red")
        self.master.after(400, lambda: self.button_A.config(bg="white"))
        
        self.move_history.append(self.output_dir_A)
        
        image_name = self.images[self.current_index]
        
        old_path = os.path.join(self.image_dir,image_name)
        new_path = os.path.join(self.output_dir_A, image_name)
        #os.rename(old_path, new_path)
        shutil.copy(old_path, new_path)
    
        self.current_index += 1
        self.load_image()

    def classify_B(self):
        self.button_B.config(bg="green")
        self.master.after(400, lambda: self.button_B.config(bg="white"))
        
        self.move_history.append(self.output_dir_B)
        
        image_name = self.images[self.current_index]
        
        old_path = os.path.join(self.image_dir,image_name)
        new_path = os.path.join(self.output_dir_B, image_name)
        #os.rename(old_path, new_path)
        shutil.copy(old_path, new_path)
        
        self.current_index += 1
        self.load_image()

    def undo(self):
        self.button_undo.config(bg="blue")
        self.master.after(400, lambda: self.button_undo.config(bg="white"))
        
        if self.move_history:
            # Get the most recent image file and its corresponding directory from the move history
            last_move = self.move_history.pop()
            
            if last_move != "":
                image_name = self.images[self.current_index - 1]

                # Move the image file back to its original directory
                old_path = os.path.join(last_move,image_name)
                #new_path = os.path.join(self.image_dir, image_name)

                #os.rename(old_path, new_path)
                os.remove(old_path)
                
            # Load the previous image
            self.current_index -= 1
            self.load_image()
            
    def skip(self):
        self.button_skip.config(bg="yellow")
        self.master.after(400, lambda: self.button_skip.config(bg="white"))
        
        self.move_history.append("")
        self.current_index += 1
        self.load_image()


In [3]:
root = tk.Tk()
root.title("Image Classifier")
image_dir = "/mnt/hgfs/shared/original_data/testing" #"/mnt/hgfs/shared/original_images/"
output_dir_A = image_dir #"data/testing"
output_dir_B = "/mnt/hgfs/shared/coco/visu" #"data/training"
app = ImageClassifier(root, image_dir, output_dir_A, output_dir_B)
root.mainloop()